# 분류

**분류**란 데이터가 어느 범주에 해당하는지 판단하는 문제로서 유사한 것들을 같은 범주로 묶으면 주변에서 일어나는 일을 빠르게 판단하는데 도움이 된다. 

## 5.1 이항분류

**이항분류**는 정답의 범주가 2가지인 문제이다.
예를 들어 개와 고양이를 분류하는 등의 가벼운 분류 문제를 다룬다.

In [ ]:
import pandas as pd
red = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep = ';')
white = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep = ';')
print(red.head())
print(white.head())

In [ ]:
red['type'] = 0
white['type']  = 1
print(red.head(2))
print(white.head(2))

wine = pd.concat([red,white])
print(wine.describe())

In [ ]:
import matplotlib.pyplot as plt
plt.hist(wine['type'])
plt.xticks([0,1])
plt.show()

print(wine['type'].value_counts())

데이터를 섞기 전에 해야할 일이 바로 '정규화'이다.

데이터의 파악을 도와주는 것이 판다스의 info()함수이다.

In [ ]:
print(wine.info())

In [ ]:
wine_norm = (wine - wine.min()) / (wine.max() - wine.min())
print(wine_norm.head())
print(wine_norm.describe())

In [ ]:
import numpy as np 
wine_shuffle = wine_norm.sample(frac=1)
print(wine_shuffle.head())
wine_np = wine_shuffle.to_numpy()
print(wine_np[:5])

판다스의 sample() 함수는 전체 데이터 프레임에서 frac 인수로 지정된 비율만큼의 행을 랜덤하게 뽑아서 새로운 데이터프레임을 만든다. frac=1로 지정됐기 때문에 100%, 즉 모든 데이터를 뽑아서 섞는 것과 동일한 효과이다. 

첫번째 출력 결과에서 type을 보면 원래는 레드 와인이 앞에 있기 때문에 0.0만 다섯개가 나와야 하지만 랜덤하게 섞였기 때문에 섞인 게 나온다. 그 다음에 나오는 to_numpy 함수는 데이터 프레임을 넘파이 array로 변환한다.



In [ ]:
import tensorflow as tf

train_idx = int(len(wine_np) * 0.8)
train_X, train_Y = wine_np[:train_idx, :-1], wine_np[:train_idx, -1]
test_X, test_Y = wine_np[train_idx:, :-1], wine_np[train_idx:, -1]

train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=2)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=2)

In [ ]:
print(train_X[0], train_Y[0], test_X[0], test_Y[0])

데이터를 훈련과 검증으로 나누기 위해서 wine_np의 수의 80%에 해당하는 인덱스를 int 값으로 저장한다. 그 다음 인덱스를 이요해 wine_np를 분리한다. 넘파이 array에서는 2차원 이상의 인덱스를 접근할 때 쉼표를 사용하기 때문에 쉼표를 이용해 X와 Y를 구분한다. 

tf.keras.utils에서 불러오는 to_categorical은 분류 문제에서 자주 쓰이는 함수로, 정답 행렬ㅇ르 원-핫 인코딩 방식으로 바꾼다. 원-핫 인코딩은 정답에 해당하는 인덱스의 값에는 1을 넣고, 나머지 인덱스에는 모두 0을 넣는 방식이다. to_categorical의 두 번째 인수인 num_classes는 정답 클래스의 개수이다. 이 문제는 이항분류 문제이기 때문에 정답은 2개이다. 

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=48, activation='relu', input_shape=(12,)),
                             tf.keras.layers.Dense(units=24, activation='relu'),
                             tf.keras.layers.Dense(units=12, activation='relu'),
                             tf.keras.layers.Dense(units=2, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07),
              loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

마지막 레이어의 활성함수로 **소프트맥스**를 사용한다.

분류에서 가장 많이 쓰이는 이 함수는 큰 값은 강조하고 작은 값은 약화하는 효과를 갖는다. 분류 네트워크의 결과값의 확률의 합이 1이되도록,,, 

마지막 레이어에서 주목할 점은 뉴런의 개수가 **2개**라는 것이다. 위에서 원-핫 인코딩으로 변환한 정답의 차원수와 동일한 값이다. 

손실 함수인 loss에서 자주쓰이던 mse 대신 **categorical_crossentropy**를 사용했다.

model.compile 함수의 metrics 인수에 정확도가 추가되었다. 분류 문제는 정확도로 퍼포먼스를 측정하기 때문에 정확도 설정은 필수이다.

In [ ]:
history = model.fit(train_X, train_Y, epochs=25, batch_size=64, validation_split=0.25)

# 여기서 에러가 떠버리는데 이유를 모르곘어요,,,, ㅠㅠㅜㅠ

## 5.2 다항분류

**다항분류**란 범주의 수가 2개를 초과하는 경우를 말한다.

In [ ]:
print(wine['quality'].describe())
print(wine['quality'].value_counts())

describe() 함수로 통계 정보를 확인

value_counts() 함수로 각 분류의 개수를 확인 가능

In [ ]:
import matplotlib.pyplot as plt

plt.hist(wine['quality'], bins=7, rwidth=0.8)
plt.show()

In [ ]:
wine.loc[wine['quality'] <= 5, 'new_quality'] = 0
wine.loc[wine['quality'] == 6, 'new_quality'] = 1
wine.loc[wine['quality'] >= 7, 'new_quality'] = 2

print(wine['new_quality'].describe())
print(wine['new_quality'].value_counts())

데이터프레임에 쓰이는 loc은 특정한 데이터의 인덱스를 골라내는 역할이다.
대괄호 안에 인수를 하나만 넣으면 행이고, 두개를 넣으면 행, 열로 골라내게된다

In [ ]:
del wine['quality']
wine_norm = (wine - wine.min()) / (wine.max() - wine.min())
wine_shuffle = wine_norm.sample(frac = 1)
wine_np = wine_shuffle.to_numpy()

train_idx = int(len(wine_np) * 0.8)
train_X, train_Y = wine_np[:train_idx, :-1], wine_np[:train_idx, -1]
test_X, test_Y = wine_np[train_idx:, :-1], wine_np[train_idx:, -1]
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes=3)
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes=3)

wine['quality']를 삭제하고 wine['new_quality']를 생성한다.

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=48, activation='relu', input_shape=(12,)),
                             tf.keras.layers.Dense(units=24, activation='relu'),
                             tf.keras.layers.Dense(units=12, activation='relu'),
                             tf.keras.layers.Dense(units=3, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_X, train_Y, epochs=25, batch_size=32, validation_split=0.25)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.ylim(0.7,1)
plt.legend()

plt.show()

In [ ]:
model.evaluate(test_X, test_Y)

## 5.3 Fashion MNIST

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_X, train_Y), (test_X, test_Y) = fashion_mnist.load_data()

print(len(train_X), len(test_X))

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(train_X[0], cmap='gray')
plt.colorbar()
plt.show()

print(train_Y[0])

matplotlib.pylot의 imshow() 함수로 이미지를 그래프의 형태로 표시할 수 있다.


In [ ]:
train_X = train_X / 255.0
test_X = test_X / 255.0

print(train_X[0])

다항 분류를 사용해볼 때는 tf.keras.utils.to_categorical를 사용하여 원-핫 인코딩으로 바꾸는 부분이 있었다.

이번에는 이를 사용하지 않을 것이다. 왜냐면 대부분의 값이 0인 희소 행렬이 되기 때문에 약간의 비효율성이 나타나기 때문이다.

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28)),
                             tf.keras.layers.Dense(units=128, activation='relu'),
                             tf.keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

원-핫인코딩이 아닌 정답 행렬을 처리하기 위해 바뀐 부분은 바로 compile 함수의 loss에 들어가는 인수이다. 여기에 앞에 썼던 categorical_crossentropy 앞에 sparse가 추가됐다. Sparse는 희소 행렬을 의미한다. 이렇게 sparse_categorical_crossentropy를 쓰면 별도의 데이터 전처리 없이 희소 행렬을 나타내는 데이터를 정답 행렬로 사용할 수 있다.

워본이 28, 28의 2차원 array이기 때문에 데이터를 일렬로 정렬하기 위해 Flatten이라는 레이러를 사용한다. 이 레이어는 다차원 데이터를 1차원으로 정리하는 역할을 한다. 다음 Dense 2개를 거쳐 예측이 출력된다. 마지막 레이어의 뉴런수는 정답 범주의 수와 같은 10개이다.

In [ ]:
history = model.fit(train_X, train_Y, epochs=25, validation_split=0.25)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.ylim(0.7,1)
plt.legend()

plt.show()

In [ ]:
model.evaluate(test_X, test_Y)

정확도가 88.3이 나왔다.